<a href="https://colab.research.google.com/github/Sebas20050700/PIPELINE_AERO-METEREOLOGICO/blob/Visual-Crossing/API_VISUAL_CROSSING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **API VISUAL CROSSING**

# **"Ingesta Pura" (Satelital)**

In [1]:
import requests
import pandas as pd
from datetime import datetime

def consultar_clima_maestro(lista_aeropuertos):
    """
    ETAPA 1: Ingesta Primaria y Generación de Coordenadas.
    Optimizado para evitar errores de tipo 'NoneType' y asegurar
    la localización en territorio peruano.
    """
    url = "https://visual-crossing-weather.p.rapidapi.com/forecast"
    api_key = "a7117854a0msh861c287680abd70p1dfc59jsnfffd314081ec"

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "visual-crossing-weather.p.rapidapi.com"
    }

    resultados = []

    print(f"--- 🛰️ INICIANDO INGESTA METEOROLÓGICA ({datetime.now().strftime('%H:%M:%S')}) ---")

    for icao in lista_aeropuertos:
        location_query = f"{icao}, Peru"

        querystring = {
            "aggregateHours": "1",
            "location": location_query,
            "contentType": "json",
            "unitGroup": "metric",
            "shortColumnNames": "false"
        }

        try:
            response = requests.get(url, headers=headers, params=querystring, timeout=10)
            response.raise_for_status()
            data = response.json()

            # Extraemos la ubicación raíz
            loc_id = list(data['locations'].keys())[0]
            loc_data = data['locations'][loc_id]
            current = loc_data.get('currentConditions', {})

            # PREDICCIÓN: Extraemos el clima que hará en la SIGUIENTE hora
            # Esto aporta valor predictivo al dashboard.
            forecast_next = loc_data.get('values', [{}])[1] if len(loc_data.get('values', [])) > 1 else {}

            # Construcción del registro con limpieza de Nulos (Fallback a 0.0)
            registro = {
                "HORA_CONSULTA": datetime.now().strftime("%H:%M:%S"),
                "aeropuerto_id": icao,
                "lat": loc_data.get('latitude'),
                "lon": loc_data.get('longitude'),
                "temp_c": current.get('temp', 0.0), # Evita el error que tuviste antes
                "viento_kmh": current.get('wspd', 0.0),
                "visibilidad_km": current.get('visibility', 10.0),
                "estado_actual": current.get('conditions', "N/D"),
                "pronostico_futuro": forecast_next.get('conditions', "Estable"),
                "fuente": "Visual Crossing API"
            }

            resultados.append(registro)
            print(f"✅ {icao}: Localizado con éxito. Clima: {registro['estado_actual']}")

        except Exception as e:
            print(f"⚠️ Error en {icao}: {e}")

    # Retornamos un DataFrame limpio
    df_final = pd.DataFrame(resultados)

    # Exportación automática para el televidente (Demo-ready)
    if not df_final.empty:
        df_final.to_csv("Reporte_etapa_clima.csv", index=False)
        print(f"\n📁 Archivo 'REPORTE_ETAPA1_CLIMA.csv' generado con {len(df_final)} registros.")

    return df_final

# --- PRUEBA DE EJECUCIÓN ---
if __name__ == "__main__":
    aeropuertos_peru = ["SPJC", "SPZO", "SPQU", "SPQT"]
    df_clima = consultar_clima_maestro(aeropuertos_peru)
    print("\n--- VISTA PREVIA DEL PRODUCTO ---")
    print(df_clima[['aeropuerto_id', 'lat', 'lon', 'temp_c', 'visibilidad_km', 'viento_kmh',  'estado_actual', 'pronostico_futuro']])

--- 🛰️ INICIANDO INGESTA METEOROLÓGICA (01:40:00) ---
✅ SPJC: Localizado con éxito. Clima: N/D
✅ SPZO: Localizado con éxito. Clima: N/D
✅ SPQU: Localizado con éxito. Clima: N/D
✅ SPQT: Localizado con éxito. Clima: N/D

📁 Archivo 'REPORTE_ETAPA1_CLIMA.csv' generado con 4 registros.

--- VISTA PREVIA DEL PRODUCTO ---
  aeropuerto_id      lat      lon  temp_c  visibilidad_km  viento_kmh  \
0          SPJC -12.0562 -77.0268     0.0            10.0         0.0   
1          SPZO -12.0562 -77.0268     0.0            10.0         0.0   
2          SPQU  40.7474 -86.0604    -5.9            16.0        23.5   
3          SPQT -12.0562 -77.0268     0.0            10.0         0.0   

  estado_actual pronostico_futuro  
0           N/D  Partially cloudy  
1           N/D  Partially cloudy  
2           N/D          Overcast  
3           N/D  Partially cloudy  


# **"Validación Cruzada" (Tierra vs. Aire)**

In [ ]:
import requests
import pandas as pd
import numpy as np

# ==============================================================================
# 1. FUNCIÓN DE CORRELACIÓN ESPACIAL (SENAMHI)
# ==============================================================================
def encontrar_estacion_cercana(lat_objetivo, lon_objetivo, archivo_csv="MAESTRO_ESTACIONES_SENAMHI_GEO.csv"):
    """
    Calcula la estación física más cercana utilizando la fórmula de distancia euclidiana
    optimizada para coordenadas geográficas.
    """
    try:
        df = pd.read_csv(archivo_csv)
    except FileNotFoundError:
        return {"ERROR": f"Archivo '{archivo_csv}' no encontrado."}

    # Evitamos errores si las coordenadas objetivo son None
    if lat_objetivo is None or lon_objetivo is None:
        return {"ERROR": "Coordenadas de origen no válidas (NaN)."}

    # Cálculo de distancia (Aproximación lineal en grados)
    # Multiplicamos por 111.12 para convertir la diferencia de grados a KM aproximados
    df['distancia_grados'] = np.sqrt((df['LATITUD'] - lat_objetivo)**2 + (df['LONGITUD'] - lon_objetivo)**2)
    df['distancia_km'] = df['distancia_grados'] * 111.12

    # Obtenemos el registro con la distancia mínima
    match = df.sort_values('distancia_km').iloc[0]

    return {
        "ESTACION": match['ESTACION'],
        "DISTANCIA_KM": round(match['distancia_km'], 2),
        "COORD": (match['LATITUD'], match['LONGITUD'])
    }

# ==============================================================================
# 2. CONSULTA API CON FILTRO DE CALIDAD
# ==============================================================================
def consultar_clima_aeropuertos(lista_aeropuertos):
    url = "https://visual-crossing-weather.p.rapidapi.com/forecast"
    api_key = "a7117854a0msh861c287680abd70p1dfc59jsnfffd314081ec"
    headers = {"x-rapidapi-key": api_key, "x-rapidapi-host": "visual-crossing-weather.p.rapidapi.com"}

    resultados = []

    for icao in lista_aeropuertos:
        location_query = f"{icao}, Peru"
        params = {"aggregateHours": "1", "location": location_query, "contentType": "json", "unitGroup": "metric"}

        print(f"🛰️ Consultando satélite para: {location_query}...")

        try:
            r = requests.get(url, headers=headers, params=params, timeout=10)
            data = r.json()
            loc_id = list(data['locations'].keys())[0]
            current = data['locations'][loc_id]['currentConditions']

            resultados.append({
                "ICAO": icao,
                "lat": data['locations'][loc_id].get('latitude'),
                "lon": data['locations'][loc_id].get('longitude'),
                "temp": current.get('temp', 0.0),
                "clima": current.get('conditions', "N/D")
            })
        except Exception as e:
            print(f"⚠️ Salto en {icao}: {e}")

    return pd.DataFrame(resultados)

# ==============================================================================
# 3. PROCESAMIENTO E INTEGRACIÓN FINAL
# ==============================================================================
if __name__ == "__main__":
    ruta_aerea = ["SPJC", "SPZO", "SPQU"]

    # Ejecución Paso 1: Satélite
    df_api = consultar_clima_aeropuertos(ruta_aerea)

    if not df_api.empty:
        print("\n" + "="*60)
        print("📊 REPORTE DE INTEGRACIÓN AERO-METEOROLÓGICA")
        print("="*60)

        datos_integrados = []

        # Ejecución Paso 2: Validación Cruzada (Tierra)
        for _, fila in df_api.iterrows():
            res_senamhi = encontrar_estacion_cercana(fila['lat'], fila['lon'])

            # Consolidamos la información
            info_vuelo = {
                "AEROPUERTO": fila['ICAO'],
                "CLIMA_SAT": fila['clima'],
                "VALIDADOR_TIERRA": res_senamhi.get("ESTACION", "⚠️ ERROR"),
                "DIST_VALIDACIÓN": f"{res_senamhi.get('DISTANCIA_KM', 0)} KM"
            }
            datos_integrados.append(info_vuelo)

        # Visualización final para los televidentes
        df_final = pd.DataFrame(datos_integrados)
        print(df_final.to_string(index=False))
        print("="*60)
    else:
        print("❌ Fallo crítico: No se recibieron datos de la API.")

🛰️ Consultando satélite para: SPJC, Peru...
🛰️ Consultando satélite para: SPZO, Peru...
🛰️ Consultando satélite para: SPQU, Peru...

📊 REPORTE DE INTEGRACIÓN AERO-METEOROLÓGICA
AEROPUERTO CLIMA_SAT VALIDADOR_TIERRA DIST_VALIDACIÓN
      SPJC       N/D         ⚠️ ERROR            0 KM
      SPZO       N/D         ⚠️ ERROR            0 KM
      SPQU       N/D         ⚠️ ERROR            0 KM
